In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
sys.path.append("/dfs/scratch1/simran/tutorial/bootleg-internal")

import numpy as np
import sklearn
from sklearn import metrics
import nltk 
from collections import Counter
import matplotlib.pyplot as plt
import itertools
import ast

nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))


### BOOTLEG ###
import sys
import load_entity_profiles

[nltk_data] Downloading package stopwords to
[nltk_data]     /lfs/1/simran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /lfs/1/simran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Loaded entity symbols.
Loading alias qid counts from /dfs/scratch0/lorr1/data_prep/data/wiki0905/stats/alias_qid_traindata_withaugment.json
Found 506 hyena types
Found 26933 wikidata types
FINISHED LOADING IN 183.5412859916687


In [98]:
from load_entity_profiles import es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count, rel_to_name, du

In [88]:
# Bootleg utility functions:
# BY ALIAS: 
def get_candidates(alias):
    try:
        # To get qid candidates of an alias
        cands = esp.get_qid_cands(alias)
        return cands
    except:
        cands = esp.get_qid_cands(alias)
    
#get_candidates('distortion')

In [115]:
# Manual User Inputs
model_0906 = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences/static_remap_embs/09262020-234021_posaware_test_rev_ent.json.csv' #baseline spanbert without bootleg
#spanbert = '/dfs/scratch1/simran/bootleg_downstream/error_outputs/0916_tacred_bert_test_predictions.csv'
spanboot_full_0926 = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences/predictions/test_predictions.txt'
spanboot = '/dfs/scratch1/simran/bootleg_downstream/error_outputs/0925_spanboot_test_predictions.csv'

In [116]:
LABEL_TO_ID = {'no_relation': 0, 'per:title': 1, 'org:top_members/employees': 2, 'per:employee_of': 3, 
               'org:alternate_names': 4, 'org:country_of_headquarters': 5, 'per:countries_of_residence': 6, 
               'org:city_of_headquarters': 7, 'per:cities_of_residence': 8, 'per:age': 9, 
               'per:stateorprovinces_of_residence': 10, 'per:origin': 11, 'org:subsidiaries': 12, 
               'org:parents': 13, 'per:spouse': 14, 'org:stateorprovince_of_headquarters': 15, 'per:children': 16, 
               'per:other_family': 17, 'per:alternate_names': 18, 'org:members': 19, 'per:siblings': 20, 
               'per:schools_attended': 21, 'per:parents': 22, 'per:date_of_death': 23, 'org:member_of': 24, 
               'org:founded_by': 25, 'org:website': 26, 'per:cause_of_death': 27, 
               'org:political/religious_affiliation': 28, 'org:founded': 29, 'per:city_of_death': 30, 
               'org:shareholders': 31, 'org:number_of_employees/members': 32, 'per:date_of_birth': 33, 
               'per:city_of_birth': 34, 'per:charges': 35, 'per:stateorprovince_of_death': 36, 'per:religion': 37, 
               'per:stateorprovince_of_birth': 38, 'per:country_of_birth': 39, 'org:dissolved': 40, 
               'per:country_of_death': 41}

LABEL_LST = list(LABEL_TO_ID.keys())

STANFORD_NER_TYPES = ['DATE', 'LOCATION', 'MONEY', 'ORGANIZATION', 'PERCENT', 'PERSON', 'TIME']
punctuation = [',', ':', '.', ';', "'", ')', '(', "'s", '--', '-', '``', "''"]

In [117]:
# Utility functions: accepts a df with corrected slices, and outputs the predicted result
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support as score
def score_corrections(df, pred_name):
    import scorer
    scorer.score(df['relation'].tolist(), df[pred_name].tolist(), verbose=True)
    df_errs = df[df['relation'] != df[pred_name]]
    print("Number of Errors: ", df_errs.shape[0])
# score_corrections(df_results)

In [118]:
# basic error rate by relation 
def error_rate_by_relation(df_r, key, df_e):
    relation_df = pd.DataFrame(columns=['relation','error_rate','error_count','total_count'])
    index = 0
    for k, v in LABEL_TO_ID.items():
        df_relation_tot = df_r[df_r[key] == k] # all rows with k = true trelation
        tot = df_relation_tot.shape[0] # number of examples with this true relation
    
        df_relation_err = df_e[df_e[key] == k] # error rows with k = true trelation
        err = df_relation_err.shape[0] # number of errors with this true relation
        error_rate = err/tot
    
        relation_df.loc[index] = pd.Series({'relation':k, 'error_rate':error_rate, 'error_count':err, 'total_count':tot})
        index += 1
    
    print(relation_df.sort_values('relation'))

In [35]:
# Load the model data 
df_results = pd.read_csv(model_iclr)
df_results.rename(columns={'prediction':'prediction_model'}, inplace=True)

# Load the spanbert data
#df_results_spanbert = pd.read_txt(spanboot_full_0926)
df_results_spanbert = pd.read_csv(spanboot_full_0926, sep="\t", header=None)
df_results_spanbert.columns = ["id","prediction"]


map_id_pred = {}
for ind, row in df_results_spanbert.iterrows():
    map_id_pred[row['id']] = row['prediction']
df_results['prediction_spanbert'] = df_results['id'].map(map_id_pred)
for key in df_results.columns.values:
    if key != 'prediction':
        df_results_spanbert[key] = df_results[key]
        
        
# Load the spanboot data
df_results_spanboot = pd.read_csv(spanboot)
map_id_pred = {}
for ind, row in df_results_spanboot.iterrows():
    map_id_pred[row['id']] = row['prediction']
df_results['prediction_spanboot'] = df_results['id'].map(map_id_pred)
for key in df_results.columns.values:
    if key != 'prediction':
        df_results_spanboot[key] = df_results[key]
        
# error rates
df_errors = df_results[df_results['relation'] != df_results['prediction_model']]
df_errors_spanbert = df_results[df_results['relation'] != df_results['prediction_spanbert']]
df_errors_spanboot = df_results[df_results['relation'] != df_results['prediction_spanboot']]


print("FULL_model size: ", df_results.shape)
print("ERRS_model size: {}\n".format(df_errors.shape))

print("FULL_baseline size: ", df_results_spanbert.shape)
print("ERRS_baseline size: {}\n".format(df_errors_spanbert.shape))

print("FULL_spanboot size: ", df_results_spanboot.shape)
print("ERRS_spanboot size: ", df_errors_spanboot.shape)



FULL_model size:  (15509, 28)
ERRS_model size: (1641, 28)

FULL_baseline size:  (15509, 28)
ERRS_baseline size: (1231, 28)

FULL_spanboot size:  (15509, 29)
ERRS_spanboot size:  (1124, 28)


In [97]:
print(df_results.columns.values)

['obj_leng' 'obj_type' 'subj_leng' 'prop_mentions' 'subj_type' 'obj'
 'obj_qids' 'separation_dist' 'example' 'real_mentions' 'subj'
 'stanford_ner' 'subj_qids' 'subj_pos' 'obj_ner' 'obj_pos' 'num_ner'
 'prop_ner' 'mentions' 'relation' 'subj_ner' 'obj_mentions' 'qids'
 'prediction_model' 'id' 'subj_mentions' 'prediction_spanbert'
 'prediction_spanboot']


## Analysis by Relation

In [37]:
#error_rate_by_relation(df_results, 'relation_model', df_errors)
relation_df = pd.DataFrame(columns=['relation','errrate_span','errcount_span', 'errrate_boot','errcount_boot','total_count'])
index = 0
for k, v in LABEL_TO_ID.items():
    if df_results_spanbert[df_results_spanbert['relation'] == k].shape[0] > 0:
        df_relation_tot_span = df_results_spanbert[df_results_spanbert['relation'] == k] # all rows with k = true trelation
        tot_span = df_relation_tot_span.shape[0] # number of examples with this true relation
    
        df_relation_err_span = df_errors_spanbert[df_errors_spanbert['relation'] == k] # error rows with k = true trelation
        err_span = df_relation_err_span.shape[0] # number of errors with this true relation
        error_rate_span = err_span/tot_span
        
        df_relation_tot_boot = df_results_spanboot[df_results_spanboot['relation'] == k] # all rows with k = true trelation
        tot_boot = df_relation_tot_boot.shape[0] # number of examples with this true relation
    
        df_relation_err_boot = df_errors_spanboot[df_errors_spanboot['relation'] == k] # error rows with k = true trelation
        err_boot = df_relation_err_boot.shape[0] # number of errors with this true relation
        error_rate_boot = err_boot/tot_boot
    
        relation_df.loc[index] = pd.Series({'relation':k, 'errrate_span':error_rate_span, 'errcount_span':err_span, 'errrate_boot':error_rate_boot,'errcount_boot': err_boot, 'total_count':tot_span})
        index += 1
    
print(relation_df.sort_values('relation'))

                               relation  errrate_span errcount_span  \
0   no_relation                          0.040045      496            
4   org:alternate_names                  0.077551      19             
7   org:city_of_headquarters             0.314607      28             
5   org:country_of_headquarters          0.478261      44             
39  org:dissolved                        0.000000      0              
29  org:founded                          0.054054      2              
25  org:founded_by                       0.144737      11             
24  org:member_of                        1.000000      4              
19  org:members                          0.937500      15             
32  org:number_of_employees/members      0.428571      6              
13  org:parents                          0.368421      21             
28  org:political/religious_affiliation  0.800000      8              
31  org:shareholders                     0.666667      2              
15  or

# Compare Model vs. Baseline Errors

We want to confirm that the improvement ties back to insights in bootleg - i.e., the mentions/types/relations in bootleg's database.

In [38]:
# get the set difference of errors
df_spanbert_only_errors = df_errors_spanbert[~df_errors_spanbert['id'].isin(df_errors_spanboot['id'])]
df_spanboot_only_errors = df_errors_spanboot[~df_errors_spanboot['id'].isin(df_errors_spanbert['id'])]
print("The #examples our model improves on vs. baseline are: ", df_spanbert_only_errors.shape[0])
print("The #examples our model does worse on vs. baseline are: ", df_spanboot_only_errors.shape[0])

# for i, row in df_spanbert_only_errors.iterrows():
#     if row['relation'] != "no_relation":
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_spanboot', 'prediction_spanbert', 'mentions', 'stanford_ner']])
#         print()
#     if i == 10:
#         break

The #examples our model improves on vs. baseline are:  515
The #examples our model does worse on vs. baseline are:  408


In [39]:
# THE DISTRIBUTION of RELATIONS WHEN MODEL IMPOVES OVER SPANBERT
print("THE DISTRIBUTION of RELATIONS WHEN MODEL IMPROVES OVER SPANBERT")
relations_errs_matchqids = []
for index, row in df_spanbert_only_errors.iterrows():
    relations_errs_matchqids.append(row['relation'])
print(Counter(relations_errs_matchqids))
print()

# THE DISTRIBUTION OF RELATIONS WHEN SPANBERT IMPROVES OVER MODEL
print(" THE DISTRIBUTION OF RELATIONS WHEN SPANBERT IMPROVES OVER MODEL")
relations_errs_matchqids = []
for index, row in df_spanboot_only_errors.iterrows():
    relations_errs_matchqids.append(row['relation'])
print(Counter(relations_errs_matchqids))


THE DISTRIBUTION of RELATIONS WHEN MODEL IMPROVES OVER SPANBERT
Counter({'no_relation': 246, 'org:top_members/employees': 42, 'per:title': 28, 'per:employee_of': 27, 'org:city_of_headquarters': 21, 'org:stateorprovince_of_headquarters': 14, 'org:country_of_headquarters': 14, 'per:origin': 13, 'per:children': 10, 'per:countries_of_residence': 9, 'per:parents': 8, 'per:cause_of_death': 8, 'per:age': 8, 'per:cities_of_residence': 8, 'org:founded_by': 7, 'per:stateorprovince_of_death': 6, 'org:website': 5, 'org:number_of_employees/members': 5, 'org:political/religious_affiliation': 5, 'per:stateorprovinces_of_residence': 5, 'org:alternate_names': 4, 'per:other_family': 4, 'per:charges': 3, 'org:subsidiaries': 3, 'per:spouse': 3, 'org:parents': 3, 'per:siblings': 2, 'per:stateorprovince_of_birth': 1, 'per:religion': 1, 'per:date_of_death': 1, 'per:city_of_death': 1})

 THE DISTRIBUTION OF RELATIONS WHEN SPANBERT IMPROVES OVER MODEL
Counter({'no_relation': 258, 'per:countries_of_residence': 

### SPANBERT WORSE THAN BOOTLEG

Here we find that the *number* of proper nouns that have errors and receive bootleg mentions are about the same w/ vs. w/out using bootleg entity embeddings. 

In [49]:
bad_qids_to_fix = []

In [46]:
# some constants
proper_noun = ['NNP', 'NNPS']
nonproper_noun = ['NN', 'NNS']
number_pos = ['CD']
nomention = ["['UNK']", "['UNK', 'UNK']", "['UNK', 'UNK', 'UNK']", "['UNK', 'UNK', 'UNK', 'UNK']", 
             "['UNK', 'UNK', 'UNK', 'UNK', 'UNK']"]
cols = ['example', 'relation','prediction_model', 'prediction_spanbert', 'prediction_spanboot', 'mentions', 'subj_pos', 'obj_pos', 'subj', 'obj', 'subj_mentions', 'subj_qids', 'obj_mentions', 'obj_qids', 'id', 'qids']

In [41]:
def get_relations(subj_qids_str, obj_qids_str):
    subj_qids = ast.literal_eval(subj_qids_str)
    obj_qids = ast.literal_eval(obj_qids_str)
    subj_qid = subj_qids[0]
    obj_qid = obj_qids[0]
    
    rels = [esp.get_relation_name(r) for r in esp.get_all_relations(subj_qid, obj_qid)]
    rels = [rel_to_name.get(rel, rel) for rel in rels]
    return rels

In [42]:
def get_related_entities(qid):
    # Get all connected qids for a given qid
    related_qids = du.get_related_qids(qid, es)
    print(f"Related QIDs {related_qids}")
    print([es.get_title(qid) for qid in related_qids])

    lst = [es.get_title(qid) for qid in related_qids]

In [50]:
# just subj and obj

bad_qids = []
bad_qids_titles = []

sub_df = df_spanbert_only_errors[cols]
for index, row in sub_df.iterrows():
    subj = row['subj']
    subj_qid = ast.literal_eval(row['subj_qids'])
    obj = row['obj']
    obj_qid = ast.literal_eval(row['obj_qids'])
    
    for qid in subj_qid:
        bad_qids.append(qid)
    for qid in obj_qid:
        bad_qids.append(qid)
        
    bad_qids_titles.append(subj)
    bad_qids_titles.append(obj)
        

print("SPANBERT DOES WORSE THAN BOOTLEG MODEL ON THESE SUBJ/OBJ titles and QIDS:")#print(len(lst_pos_subj_spanbert))
worst_for_spanbert_qids = Counter(bad_qids).most_common(100)
print(worst_for_spanbert_qids)

print()
worst_for_spanbert_titles = Counter(bad_qids_titles).most_common(100)
print(worst_for_spanbert_titles)
worst_for_spanbert = [tup[0] for tup in worst_for_spanbert]

for tup in worst_for_spanbert_qids:
    bad_qids_to_fix.append(tup[0])

SPANBERT DOES WORSE THAN BOOTLEG MODEL ON THESE SUBJ/OBJ titles and QIDS:
[('UNK', 409), ('Q599439', 41), ('Q1186190', 40), ('Q2498389', 36), ('Q6978094', 36), ('Q1967040', 30), ('Q930069', 29), ('Q505624', 28), ('Q7927961', 26), ('Q459256', 22), ('Q777636', 20), ('Q4379875', 16), ('Q1137260', 16), ('Q736402', 16), ('Q1888308', 15), ('Q179202', 15), ('Q7039832', 14), ('Q7461226', 12), ('Q6990032', 12), ('Q148', 11), ('Q20709176', 11), ('Q219569', 11), ('Q351280', 11), ('Q957941', 10), ('Q30', 9), ('Q24534740', 9), ('Q906722', 9), ('Q456536', 9), ('Q487907', 9), ('Q4466010', 9), ('Q3161542', 9), ('Q4888392', 8), ('Q27718550', 8), ('Q6832291', 8), ('Q200106', 8), ('Q20740852', 8), ('Q3540951', 7), ('Q607131', 7), ('Q1008', 7), ('Q180962', 7), ('Q843', 7), ('Q6172037', 6), ('Q1211', 6), ('Q5702577', 6), ('Q11725', 6), ('Q49201', 6), ('Q62015529', 6), ('Q5204240', 6), ('Q20711120', 6), ('Q24534585', 6), ('Q1990987', 6), ('Q465140', 6), ('Q4739665', 6), ('Q322456', 5), ('Q16555', 5), ('Q853

In [ ]:
df_spanbert_only_errors

In [51]:
# full sentence

bad_qids_all = []

sub_df = df_spanbert_only_errors[cols]
for index, row in sub_df.iterrows():
    qids = ast.literal_eval(row['qids'])
    for qid in qids:
        bad_qids_all.append(qid)

print("SPANBERT DOES WORSE THAN BOOTLEG MODEL ON THESE SUBJ/OBJ titles and QIDS:")#print(len(lst_pos_subj_spanbert))
worst_for_spanbert_qids_all = Counter(bad_qids_all).most_common(100)
print(worst_for_spanbert_qids_all)

for tup in worst_for_spanbert_qids_all:
    bad_qids_to_fix.append(tup[0])

SPANBERT DOES WORSE THAN BOOTLEG MODEL ON THESE SUBJ/OBJ titles and QIDS:
[('UNK', 12588), ('Q1186190', 73), ('Q6172037', 56), ('Q599439', 41), ('Q30', 36), ('Q930069', 36), ('Q2498389', 36), ('Q6978094', 36), ('Q7091120', 34), ('Q541282', 34), ('Q607131', 34), ('Q258', 32), ('Q332454', 30), ('Q148', 30), ('Q1967040', 30), ('Q459256', 28), ('Q6581823', 28), ('Q505624', 28), ('Q11725', 27), ('Q7927961', 26), ('Q56633627', 26), ('Q60', 25), ('Q6174161', 24), ('Q180962', 23), ('Q1929568', 22), ('Q484876', 22), ('Q250656', 22), ('Q6757146', 21), ('Q7728721', 20), ('Q777636', 20), ('Q1888308', 19), ('Q179202', 18), ('Q27718550', 18), ('Q140686', 18), ('Q49201', 18), ('Q7039832', 18), ('Q28183555', 18), ('Q1137260', 17), ('Q1008', 17), ('Q219569', 17), ('Q4379875', 16), ('Q212235', 16), ('Q55', 16), ('Q6990032', 16), ('Q5466658', 16), ('Q7248108', 16), ('Q192283', 16), ('Q736402', 16), ('Q332209', 15), ('Q235517', 15), ('Q5544196', 15), ('Q537842', 15), ('Q51490', 15), ('Q1951543', 14), ('Q9

In [52]:
# Inspecting examples where bootleg had any relation for the subj, obj pair

count = 0
for index, row in df_spanbert_only_errors.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count += 1
print("For ", count, " spanbert only errors, bootleg has *some* relation between the subj and obj")

count = 0
for index, row in df_spanbert_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and not any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_spanboot', 'prediction_spanbert', 'mentions', 'stanford_ner']])
#         print(f"Rels {rels}")
#         print()
        count += 1

print("For ", count, " spanbert only errors, the existing bootleg relation is NOT a subset of the gold relation")


For  49  spanbert only errors, bootleg has *some* relation between the subj and obj
For  39  spanbert only errors, the existing bootleg relation is NOT a subset of the gold relation


In [53]:
# alternate names
alternate_name_errs = df_spanbert_only_errors[df_spanbert_only_errors['relation'].str.contains("alternate")].shape
print("For ", alternate_name_errs[0], " spanbert errors, the gold label is ALTERNATE NAMES ")


For  4  spanbert errors, the gold label is ALTERNATE NAMES 


In [57]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanbert_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
        print(f"Rels {rels}")
        print(row['relation'])
        print(row['prediction_spanbert'])
        print()
        count += 1
        
print("For ", count, " spanbert errors, the bootleg relation is a subset (or the same) as the gold relation")


Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

Rels ['country']
org:country_of_headquarters
no_relation

For  10  spanbert errors, the bootleg relation is a subset (or the same) as the gold relation


In [58]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanbert_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['prediction_spanbert']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " spanbert errors, the bootleg relation is a subset (or the same) as the spanbert prediction")


For  2  spanbert errors, the bootleg relation is a subset (or the same) as the spanbert prediction


### FOR EXAMPLES WHERE SPANBERT IS BETTER THAN BOOTLEG

In [59]:
lst_pos_subj_model = []
lst_subj_nomention_model = {}
lst_pos_obj_model = []
lst_obj_nomention_model= {}

sub_df = df_spanboot_only_errors[cols]
for index, row in sub_df.iterrows():
    if any(pos in row['subj_pos'] for pos in proper_noun): # is the subj a proper noun?
        lst_pos_subj_model.append(row['subj'])
        if any(null in row['subj_mentions'] for null in nomention): # is the bootleg mention empty for this proper noun?
            lst_subj_nomention_model[row['id']] = row['subj']
            
    if any(pos in row['obj_pos'] for pos in proper_noun):
        lst_pos_obj_model.append(row['obj'])
        if any(null in row['obj_mentions'] for null in nomention):
            lst_obj_nomention_model[row['id']] = row['obj']
        
print("The number of proper noun subj in bootleg (&not spanbert) errors are:", len(lst_pos_subj_spanbert), "and obj are:", len(lst_pos_obj_spanbert))
print("The number of proper noun subj in bootleg (&not spanbert) errors that don't get a bootleg mention are:", len(lst_subj_nomention_spanbert.keys()), "and obj are:", len(lst_obj_nomention_spanbert.keys()))
print()

lst_pos_subj_model.extend(lst_pos_obj_model)
print("SPANBERT DOES BETTER THAN BOOTLEG MODEL ON THESE SUBJ/OBJ PROPER NOUNS")#print(len(lst_pos_subj_spanbert))
worst_for_model = Counter(lst_pos_subj_model).most_common(50)
print(worst_for_model)
worst_for_model = [tup[0] for tup in worst_for_model]

The number of proper noun subj in bootleg (&not spanbert) errors are: 742 and obj are: 335
The number of proper noun subj in bootleg (&not spanbert) errors that don't get a bootleg mention are: 20 and obj are: 26

SPANBERT DOES BETTER THAN BOOTLEG MODEL ON THESE SUBJ/OBJ PROPER NOUNS
[("['alico']", 15), ("['ruben', 'van', 'assouw']", 13), ("['corporate', 'library']", 12), ("['china']", 11), ("['wen', 'qiang']", 9), ("['xie', 'caiping']", 8), ("['matteo', 'messina', 'denaro']", 7), ("['chen', 'tao']", 7), ("['menil', 'collection']", 7), ("['national', 'development', 'and', 'reform', 'commission']", 7), ("['ruben']", 7), ("['us']", 7), ("['countrywide']", 6), ("['national', 'congress', 'of', 'american', 'indians']", 6), ("['norris', 'church', 'mailer']", 6), ("['gross']", 6), ("['asia', 'bibi']", 6), ("['new', 'fabris']", 6), ("['samudio']", 6), ("['u.s.']", 6), ("['graham']", 5), ("['ahmed', 'rashid']", 5), ("['wen']", 5), ("['amanda', 'knox']", 5), ("['washington']", 5), ("['cuba']", 5

In [60]:
# model_worse_than_spanbert = [tup for tup in worst_for_model if tup not in worst_for_spanbert]
# for item in model_worse_than_spanbert:
#     item = ast.literal_eval(item)
#     item = ' '.join(item)
#     cands = get_candidates(item)
#     print("Cands for ", item, " are: ", f"Cands {cands}", "\n")

In [61]:
sub_df = df_spanboot_only_errors[cols] #BOOTLEG ERRORS, and NOT BASELINE ERRORS
count_missed_propernoun = 0
count_missed_propernoun_with_bootrels = 0
ids_missed_propernoun_with_bootrels = []
for index, row in sub_df.iterrows():
    if any(pos in row['subj_pos'] for pos in proper_noun) or any(pos in row['obj_pos'] for pos in proper_noun):
#        print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
        rels = get_relations(row['subj_qids'], row['obj_qids'])
        if rels:
            count_missed_propernoun_with_bootrels += 1
            ids_missed_propernoun_with_bootrels.append(row['id'])
#         print(f"Rels {rels}")
#         print()
        count_missed_propernoun += 1
        
print("Number of examples where bootleg had a relation for the subj, obj pair: ", count_missed_propernoun_with_bootrels)
print("Number of examples where subj and/or obj is a proper noun: ", count_missed_propernoun)


Number of examples where bootleg had a relation for the subj, obj pair:  44
Number of examples where subj and/or obj is a proper noun:  361


In [62]:
# Inspecting examples where bootleg had any relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanboot_only_errors.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count += 1
print("For ", count, " bootleg errors, bootleg has *some* relation between the subj and obj\n")


print("For these bootleg errors, the existing bootleg relation is NOT a string-subset of the gold relation")
count = 0
for index, row in df_spanboot_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and not any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1

print("For ", count, " bootleg errors, the existing bootleg relation is NOT a subset of the gold relation")


For  44  bootleg errors, bootleg has *some* relation between the subj and obj

For these bootleg errors, the existing bootleg relation is NOT a string-subset of the gold relation
For  43  bootleg errors, the existing bootleg relation is NOT a subset of the gold relation


In [63]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanboot_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " bootleg errors, the bootleg relation is a subset (or the same) as the gold relation")


For  1  bootleg errors, the bootleg relation is a subset (or the same) as the gold relation


In [64]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['prediction_model']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " bootleg errors, the bootleg relation is a subset (or the same) as the bootleg prediction")


For  4  bootleg errors, the bootleg relation is a subset (or the same) as the bootleg prediction


#### For these proper nouns that don't get a bootleg mention, is it because they are not in bootleg's database?

In [65]:
#print(lst_subj_nomention_model)

no_candidates_values = []
has_candidates_values = []
# alias is the subj/obj that did not receive a bootleg mention
for idx, value in lst_subj_nomention_model.items():
    row = df_results[df_results['id'] == idx]   
    value = ast.literal_eval(value)
    alias = " ".join(value)
    try:
        cands = esp.get_qid_cands(alias)
        #print("{}\n{}\n{}\n\n".format(row['example'], alias, f"Cands {cands}") # uncomment to view the alias in context downstream
        has_candidates_values.append(alias)
    except: 
        no_candidates_values.append(alias)

print("Of these values that received no bootleg mention,", len(has_candidates_values), "(total)/", len(set(has_candidates_values)), "(unique), ARE IN the bootleg database.")

######## THIS SET OF ENTITIES SIMPLY DON'T HAVE CANDIDATES FOR THEM IN BOOTLEG! WE SHOULD ADD THEM! ############
print("Of these values that received no bootleg mention,", len(no_candidates_values), "(total)/", len(set(no_candidates_values)), "(unique), ARE NOT IN the bootleg database.")
print()
print(set(no_candidates_values))


Of these values that received no bootleg mention, 0 (total)/ 0 (unique), ARE IN the bootleg database.
Of these values that received no bootleg mention, 11 (total)/ 5 (unique), ARE NOT IN the bootleg database.

{'ladyzhenskiy', 'uasr', 'tamaihia', 'shoppertrak', 'cerniglia'}


In [66]:
# TODO: whether check the type of the subject and object to see if it aligns wiht the relation. 
# e.g. PARIS went to PARIS -- do the types of the entities TYPECHECK (i.e. person, location, organization) -- i.e. a location cannot go to an organzation 
# if "X" in sentence, it cannot be "Y" relation -- we want to do this in a systematic way


# If I fix this bucket, what score would I expect. Modular function. -- 

#### Error rates for words

In [67]:
nomention_model_not_spanbert = []
sub_df = df_spanbert_only_errors[cols]
for index, row in sub_df.iterrows():
    qids = row['mentions']
    qids = ast.literal_eval(qids)
    words = row['example'].split(' ')
    for i in range(len(words)):
        if qids[i] == 'UNK' and words[i] not in stop_words and words[i] not in punctuation:
            nomention_model_not_spanbert.append(words[i])

print("WORSE FOR SPANBERT THAN MODEL")
ctr_nomention = Counter(nomention_model_not_spanbert).most_common(50)
print(ctr_nomention)


nomention_spanbert_or_model = []
sub_df = df_spanboot_only_errors[cols]
for index, row in sub_df.iterrows():
    qids = row['mentions']
    qids = ast.literal_eval(qids)
    words = row['example'].split(' ')
    for i in range(len(words)):
        if qids[i] == 'UNK' and words[i] not in stop_words and words[i] not in punctuation:
            nomention_spanbert_or_model.append(words[i])
            
print("WORSE FOR MODEL THAN SPANBERT")
print(Counter(nomention_spanbert_or_model).most_common(50))


WORSE FOR SPANBERT THAN MODEL
[('said', 134), ('-rrb-', 65), ('church', 59), ('-lrb-', 56), ('wife', 43), ('former', 42), ('years', 34), ('also', 31), ('director', 28), ('wives', 28), ('one', 25), ('new', 25), ('firm', 24), ('two', 23), ('living', 21), ('death', 21), ('company', 21), ('research', 20), ('40', 20), ('brother', 20), ('news', 20), ('father', 19), ('newspaper', 18), ('41', 18), ('last', 18), ('mother', 17), ('told', 16), ('sixth', 16), ('three', 16), ('married', 16), ('addition', 15), ('granddaughter', 15), ('country', 15), ('son', 15), ('11', 15), ('says', 14), ('boy', 14), ('head', 14), ('according', 14), ('late', 13), ('could', 13), ('first', 13), ('another', 12), ('senior', 12), ('chief', 12), ('year', 12), ('member', 11), ('get', 11), ('accused', 11), ('like', 11)]
WORSE FOR MODEL THAN SPANBERT
[('said', 98), ('-rrb-', 53), ('-lrb-', 48), ('two', 29), ('years', 26), ('also', 22), ('new', 20), ('billion', 19), ('one', 19), ('former', 18), ('mother', 17), ('firm', 16), (

In [68]:
errates_model = {}
for index, row in df_results.iterrows():
    err = 0
    if row['relation'] != row['prediction_spanboot']:
        err = 1
    words = row['example'].split(' ')
    for i in range(len(words)):
        if words[i] not in stop_words and words[i] not in punctuation:
            if words[i] in errates_model:
                errates_model[words[i]]['count'] += 1
                errates_model[words[i]]['errs'] += err
            else:
                errates_model[words[i]] = {}
                errates_model[words[i]]['count'] = 1
                errates_model[words[i]]['errs'] = err

for k, v in errates_model.items():
    errates_model[k]['errrate'] = errates_model[k]['errs']/errates_model[k]['count'] 
    if errates_model[k]['errs'] > 10:
        print(k, v)


policy {'count': 135, 'errs': 15, 'errrate': 0.1111111111111111}
late {'count': 276, 'errs': 23, 'errrate': 0.08333333333333333}
u.s. {'count': 426, 'errs': 26, 'errrate': 0.06103286384976526}
alan {'count': 186, 'errs': 25, 'errrate': 0.13440860215053763}
national {'count': 1342, 'errs': 77, 'errrate': 0.05737704918032787}
director {'count': 712, 'errs': 51, 'errrate': 0.07162921348314606}
co-founder {'count': 109, 'errs': 11, 'errrate': 0.10091743119266056}
progressive {'count': 139, 'errs': 12, 'errrate': 0.08633093525179857}
democrats {'count': 132, 'errs': 11, 'errrate': 0.08333333333333333}
america {'count': 353, 'errs': 33, 'errrate': 0.09348441926345609}
member {'count': 255, 'errs': 25, 'errrate': 0.09803921568627451}
international {'count': 459, 'errs': 48, 'errrate': 0.10457516339869281}
messina {'count': 153, 'errs': 22, 'errrate': 0.1437908496732026}
denaro {'count': 180, 'errs': 26, 'errrate': 0.14444444444444443}
new {'count': 1108, 'errs': 111, 'errrate': 0.100180505415

In [70]:
errates_model = {}
for index, row in df_results.iterrows():
    err = 0
    if row['relation'] != row['prediction_spanbert']:
        err = 1
    words = row['example'].split(' ')
    for i in range(len(words)):
        if words[i] not in stop_words and words[i] not in punctuation:
            if words[i] in errates_model:
                errates_model[words[i]]['count'] += 1
                errates_model[words[i]]['errs'] += err
            else:
                errates_model[words[i]] = {}
                errates_model[words[i]]['count'] = 1
                errates_model[words[i]]['errs'] = err

for k, v in errates_model.items():
    errates_model[k]['errrate'] = errates_model[k]['errs']/errates_model[k]['count'] 
    if errates_model[k]['errs'] > 10:
        print(k, v)

policy {'count': 135, 'errs': 15, 'errrate': 0.1111111111111111}
late {'count': 276, 'errs': 29, 'errrate': 0.10507246376811594}
u.s. {'count': 426, 'errs': 20, 'errrate': 0.046948356807511735}
alan {'count': 186, 'errs': 28, 'errrate': 0.15053763440860216}
national {'count': 1342, 'errs': 91, 'errrate': 0.06780923994038748}
director {'count': 712, 'errs': 66, 'errrate': 0.09269662921348315}
co-founder {'count': 109, 'errs': 13, 'errrate': 0.11926605504587157}
progressive {'count': 139, 'errs': 11, 'errrate': 0.07913669064748201}
america {'count': 353, 'errs': 30, 'errrate': 0.08498583569405099}
member {'count': 255, 'errs': 30, 'errrate': 0.11764705882352941}
international {'count': 459, 'errs': 46, 'errrate': 0.10021786492374728}
department {'count': 241, 'errs': 13, 'errrate': 0.05394190871369295}
messina {'count': 153, 'errs': 20, 'errrate': 0.13071895424836602}
denaro {'count': 180, 'errs': 23, 'errrate': 0.12777777777777777}
new {'count': 1108, 'errs': 116, 'errrate': 0.104693140

# Just Dig into Model Errors

### For models that used manual aliases, what happened on alias-examples

In [71]:
aliases_to_add = {}
aliases_to_add['Millipore'] = {'alias':'merck millipore', 'qid':'Q1669719', 'len':1}
aliases_to_add['US '] = {'alias':'united states', 'qid':'Q30', 'len':1}
aliases_to_add['U.S.'] = {'alias':'united states', 'qid':'Q30', 'len':1}
aliases_to_add['WNO'] = {'alias':"washington national opera", 'qid':'Q386613', 'len':1}

for k, v in errates_model.items():
    for alias, content in aliases_to_add.items():
        if alias.lower() in k:  
            if k in errates_spanbert:
                spanbert_error_ct = errates_spanbert[k]['errs']
            else:
                spanbert_error_ct = 0
            spanboot_error_ct = errates_model[k]['errs']
            print("{}: spanbert_errs {}; spanboot_errs {}".format(k, spanbert_error_ct, spanboot_error_ct))
            

u.s.: spanbert_errs 20; spanboot_errs 20
china-u.s.: spanbert_errs 0; spanboot_errs 0
u.s.dollars: spanbert_errs 0; spanboot_errs 0


In [72]:
## Lots of pronoun stuff

In [137]:
for ind, row in df_errors_spanbert.iterrows():
    if ' he ' in row['example']:
        print(row[cols])
        print()

example                he formed his own company , tau , for a while , sold part to trimble , left trimble to start a gps business line at motorola -lrb- where he was also a vp -rrb- , from there to rand mcnally in chicago , and more recently to navteq .                                                                                                                                                                                                                            
relation               per:employee_of                                                                                                                                                                                                                                                                                                                                                                                                                                                    
prediction_model       no_relation                

Name: 8522, dtype: object

example                luciano ghirga , a lawyer for knox , said he had sought to have the american placed under house arrest .                                                                      
relation               per:title                                                                                                                                                                     
prediction_model       no_relation                                                                                                                                                                   
prediction_spanbert    no_relation                                                                                                                                                                   
prediction_spanboot    no_relation                                                                                                                                                   

example                `` but i would be wary of saying he was behind the kidnapping and murder of the polish engineer , '' he added .                                                                                                                        
relation               no_relation                                                                                                                                                                                                                            
prediction_model       per:charges                                                                                                                                                                                                                            
prediction_spanbert    no_relation                                                                                                                                                                                                         

In [144]:
count = 0
ids_spanboot = []
pronouns = ['he', 'she', 'her', 'his', 'him']
for index, row in df_errors_spanboot.iterrows():
    subj = ast.literal_eval(row['subj'])
    obj = ast.literal_eval(row['obj'])
    if any(s for s in subj if s in pronouns) or any(o for o in obj if o in pronouns):
        print(row[cols])
        ids_spanboot.append(row['id'])
        count += 1
        
count
      


example                she moved to sao paulo , where the baby , a boy , was born in february .                                                                                                                                        
relation               no_relation                                                                                                                                                                                                     
prediction_model       no_relation                                                                                                                                                                                                     
prediction_spanbert    per:date_of_birth                                                                                                                                                                                               
prediction_spanboot    per:date_of_birth                                

example                adi amit , the fort lauderdale lawyer , filed a lawsuit after piedra 's practice charged thousands to a credit card without his permission and then would n't refund it .                                                                                                                                      
relation               no_relation                                                                                                                                                                                                                                                                                                    
prediction_model       per:title                                                                                                                                                                                                                                                                                                      
prediction_spanbert

example                he is survived by his son , terry , of temecula , calif. ; his daughters , carol daniels of morro bay , calif. , and julie jones of la mesa , calif. ; three grandchildren ; and a great-granddaughter .                                                                                                                                                                                                                                                           
relation               per:parents                                                                                                                                                                                                                                                                                                                                                                                                                                                        
prediction_model       no_relation                

265

In [ ]:
count = 0
ids_spanboot = []
for index, row in df_errors_spanboot.iterrows():
    if row['relation'] == 'per:age':
        row_spanbert = df_errors_spanbert[row['id']].item()
        row_spanbert['pred']
            
        
count

In [145]:
count = 0
ids_spanbert = []
pronouns = ['he', 'she', 'her', 'his', 'him']
for index, row in df_errors_spanbert.iterrows():
    subj = ast.literal_eval(row['subj'])
    obj = ast.literal_eval(row['obj'])
    if any(s for s in subj if s in pronouns) or any(o for o in obj if o in pronouns):
        # print(row[cols])
        ids_spanbert.append(row['id'])
        count += 1
        
count
      

286

In [149]:
print(len(list(set(ids_spanboot) - set(ids_spanbert))))
print(len(list(set(ids_spanbert) - set(ids_spanboot))))

99
120


In [ ]:
# What is the error rate by bootleg relation?

# Tacred paper insights

### For different relations, what is the error rate with and without using Bootleg embeddings?

In [74]:
count_boot = 0
for index, row in df_errors_spanboot.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count_boot += 1
        
count_bert = 0
for index, row in df_errors_spanbert.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count_bert += 1
        
print("For {}/{} errors, spanboot has *some* relation between the subj and obj\n".format(count_boot, df_errors_spanboot.shape[0]))
print("For {}/{} errors, spanbert has *some* relation between the subj and obj\n".format(count_bert, df_errors_spanbert.shape[0]))
print("Relative: ", count_bert/count_boot)

For 122/1124 errors, spanboot has *some* relation between the subj and obj

For 127/1231 errors, spanbert has *some* relation between the subj and obj

Relative:  1.040983606557377


In [94]:
import ujson
def load_json_file(filename):
    with open(filename, 'r') as f:
        contents = ujson.load(f)
    return contents

In [107]:
model_base = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_downloads/emb_data/'
typename2title_wd = load_json_file("{}wikidatatypeid_to_title_0905.json".format(model_base))

def get_type_ids(df):
    
    # analysis structures
    missed_types = []
    row_ids_subj = []
    row_ids_obj = []
    exists_subj = 0
    exists_obj = 0
    not_exists = 0
    
    for ind, row in df.iterrows():
        subj_qids = ast.literal_eval(row['subj_qids'])
        obj_qids = ast.literal_eval(row['obj_qids'])
        idx = row['id']
        

        # get the types list for the entity qid
        try:
            types_subj = du.get_qid_types(subj_qids[0], esp, typeid2typename_wd, wikidata=True)
        except:
            types_subj = []

        try:
            types_obj = du.get_qid_types(obj_qids[0], esp, typeid2typename_wd, wikidata=True)
        except:
            types_obj = []
        #print(types_subj)
                     
            
        if row['relation'] != row['prediction_spanbert']:
            missed_types.append(types_subj)
            missed_types.append(types_obj)
        
        # get the first type in the types list   
        if types_subj:
            type_subj_qid = types_subj[0] 
            row_ids_subj.append(idx)
            exists_subj +=1
        else:
            type_subj_id = 0
            not_exists += 1

        if types_obj:
            type_obj_qid = types_obj[0] 
            row_ids_obj.append(idx)
            exists_obj +=1
        else:
            type_obj_id = 0
            not_exists += 1

    print("The number of subjects with a type relation: ", exists_subj)
    print("The number of objects with a type relation: ",exists_obj)   
    
    return row_ids_subj, row_ids_obj, missed_types

In [108]:
import json
typeqid_to_id = json.load(open("/dfs/scratch0/lorr1/bootleg/embs/wikidata_to_typeid.json"))
row_ids_subj, row_ids_obj, missed_types = get_type_ids(df_results)

The number of subjects with a type relation:  8684
The number of objects with a type relation:  9221


In [113]:
# print(missed_types)
list_missed_types = []
for tup in missed_types:
    for i in range(len(tup)):
        list_missed_types.append(typename2title_wd[tup[i]])
print(len(list_missed_types))
print(len(set(list_missed_types)))

3362
537


In [114]:
print(Counter(list_missed_types).most_common(100))


[('country', 152), ('politician', 148), ('sovereign state', 146), ('journalist', 119), ('business', 110), ('writer', 96), ('big city', 80), ('actor', 78), ('city of the United States', 73), ('film actor', 62), ('constitutional republic', 59), ('city', 48), ('enterprise', 47), ('stage actor', 43), ('special library', 42), ('city with millions of inhabitants', 39), ('credit institution', 35), ('lawyer', 34), ('military personnel', 33), ('painter', 31), ('organization', 31), ('singer', 27), ('film director', 27), ('human rights activist', 26), ('nonprofit organization', 25), ('activist', 24), ('television actor', 23), ('position', 23), ('author', 23), ('diplomat', 22), ('unitary state', 22), ('profession', 21), ('state of the United States', 21), ('military officer', 21), ('ministry of the State Council', 20), ('film producer', 20), ('aircraft pilot', 19), ('trade association', 19), ('criminal', 18), ('company', 18), ('association football player', 18), ('direct-controlled municipality', 

In [103]:
subj_df = df_results[df_results['id'].isin(row_ids_subj)]

tot_subj = subj_df.shape
spanbert_errs = 0
spanboot_errs = 0
missed_types = []
for ind, row in subj_df.iterrows():
    if row['relation'] != row['prediction_spanbert']:
        spanbert_errs += 1
        
    if row['relation'] != row['prediction_spanboot']:
        spanboot_errs += 1
        
print("For {}/{} subj_df examples, spanboot makes an error".format(spanboot_errs, tot_subj))
print("For {}/{} subj_df examples, spanbert makes an error".format(spanbert_errs, tot_subj))


For 719/(8684, 28) subj_df examples, spanboot makes an error
For 819/(8684, 28) subj_df examples, spanbert makes an error


In [104]:
obj_df = df_results[df_results['id'].isin(row_ids_obj)]

tot_obj = obj_df.shape
spanbert_errs = 0
spanboot_errs = 0
for ind, row in obj_df.iterrows():
    if row['relation'] != row['prediction_spanbert']:
        spanbert_errs += 1
    if row['relation'] != row['prediction_spanboot']:
        spanboot_errs += 1
        
print("For {}/{} obj_df examples, spanboot makes an error".format(spanboot_errs, tot_obj))
print("For {}/{} obj_df examples, spanbert makes an error".format(spanbert_errs, tot_obj))


For 861/(9221, 28) obj_df examples, spanboot makes an error
For 930/(9221, 28) obj_df examples, spanbert makes an error


In [105]:
# either the subj/obj/both have a type!

all_subj_and_obj_ids = row_ids_subj
all_subj_and_obj_ids.extend(row_ids_obj) 
all_subj_and_obj_ids = set(all_subj_and_obj_ids)
all_type_df = df_results[df_results['id'].isin(all_subj_and_obj_ids)]

tot_type = all_type_df.shape
spanbert_errs = 0
spanboot_errs = 0
for ind, row in all_type_df.iterrows():
    if row['relation'] != row['prediction_spanbert']:
        spanbert_errs += 1
    if row['relation'] != row['prediction_spanboot']:
        spanboot_errs += 1
        
print("For {}/{} examples with a subj/obj type, spanboot makes an error".format(spanboot_errs, tot_type))
print("For {}/{} examples with a subj/obj type, spanbert makes an error".format(spanbert_errs, tot_type))
print("Error Rate SpanBERT/SpanBOOT = {}".format((spanbert_errs/tot_type[0])/(spanboot_errs/tot_type[0])))

For 1022/(12299, 28) examples with a subj/obj type, spanboot makes an error
For 1128/(12299, 28) examples with a subj/obj type, spanbert makes an error
Error Rate SpanBERT/SpanBOOT = 1.1037181996086105
